In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importando Bibliotecas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.neural_network import MLPClassifier

In [ ]:
# Leitura dos dados

In [ ]:
df_train = pd.read_csv('../input/mini-flight-delay-prediction/flight_delays_train.csv', engine="python")

In [ ]:
df_test = pd.read_csv('../input/mini-flight-delay-prediction/flight_delays_test.csv')

# **Tyding Data** | Train

In [ ]:
df_train.head()

In [ ]:
# Função para Tydi
def to_int(entr):
  return int(str(entr).replace('c-', ''))

def to_bool(value):
  if(value == 'N'):
    return 1
  return 0

def categorizer(x, library):
  if(x not in library):
    library[x] = len(library)
  return library[x]

In [ ]:
# Conversão das colunas Month, DayofMonth, DayOfWeek, DepTime e Distance para inteiro
for i in ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'Distance']:
  df_train[i] = df_train[i].map(lambda x : to_int(x))

# Conversão da coluna dep_delayed_15min para inteiro booleano
df_train.dep_delayed_15min = df_train.dep_delayed_15min.map(lambda x: to_bool(x))

# Categorizando a coluna UniqueCarrier em valores em inteiros
unique_cat = {}
df_train.UniqueCarrier = df_train.UniqueCarrier.map(lambda x: categorizer(x, unique_cat))

# Categorizando as colunas Origin e Dest em valores em inteiros
airports = {}
for i in ['Origin', 'Dest']:
  df_train[i] = df_train[i].map(lambda x : categorizer(x, airports))

df_train.dtypes

# Tyding Data | Test

In [ ]:
df_test.head()

In [ ]:
# Conversão das colunas Month, DayofMonth, DayOfWeek, DepTime e Distance para inteiro
for i in ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'Distance']:
  df_test[i] = df_test[i].map(lambda x : to_int(x))

# Categorizando a coluna UniqueCarrier em valores em inteiros
df_test.UniqueCarrier = df_test.UniqueCarrier.map(lambda x: categorizer(x, unique_cat))

# Categorizando as colunas Origin e Dest em valores em inteiros
for i in ['Origin', 'Dest']:
  df_test[i] = df_test[i].map(lambda x : categorizer(x, airports))

df_test.dtypes

# Classificação

In [ ]:
labels = df_train.dep_delayed_15min

del df_train['dep_delayed_15min'] #Remoção da coluna de resultados

feat = df_train

In [ ]:
train, test, train_labels, test_labels = train_test_split(feat, labels, test_size=0.5, random_state=45)

In [ ]:
MLP = MLPClassifier(hidden_layer_sizes=(500, 20), max_iter=3000)
MLP.fit(train, train_labels)

predicted_labels = MLP.predict(test)

In [ ]:
print('Precisão: ', metrics.f1_score(test_labels, predicted_labels))

# Exportação da análise

In [ ]:
mlp_output = MLPClassifier(hidden_layer_sizes=(500, 20), max_iter=3000)
mlp_output.fit(df_train, labels)

output = mlp_output.predict(df_test)

In [ ]:
result = []
for i in output:
  if i:
    result.append("N")
  else:
    result.append("S")

df_final = pd.DataFrame(data=result)
df_final.to_csv('submission.csv', index=False)